In [8]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
from bs4 import BeautifulSoup
import requests
import csv
import os

In [9]:
import sys
sys.path.insert(0, '/aclImdb/')

In [10]:
os.getcwd()

'/Users/Stephen/Documents/BeastMode'

Let's define a function that takes a URL for an IMDB movie page, scrapes the URL, and returns the name of the movie. 

In [84]:
def get_movie(url):
    pageText = requests.get(url)
    while (pageText == None):
        time.sleep(5)
        pageText = requests.get(url)
    soup = BeautifulSoup(pageText.text,"html.parser")
    if soup == None or soup.find("div",attrs={"id":"tn15title"}) == None:
        return None
    return soup.find("div",attrs={"id":"tn15title"}).find("a").get_text()

In [67]:
# get all pos urls for the TRAIN set
with open('aclImdb/train/urls_pos.txt', 'r') as f:
    train_pos_urls = f.readlines()
    print len(train_pos_urls)

12500


In [68]:
# get all positive urls for the TEST set
with open('aclImdb/test/urls_pos.txt', 'r') as f:
    test_pos_urls = f.readlines()
    print len(train_pos_urls)

12500


Let's make a dictionary of of (URL: Movie) pairs for each URL in the training positive list of urls. This dictionary will be of use later. It is better to do all the scraping up front. (The IMDB data set did not provide us with the names of the movies, only the URLS.)

In [ ]:
% time
import random
# Make a dictionary of URL: movie name
url_dict = dict(zip(train_pos_urls, [None]*len(train_pos_urls)))
for url in train_pos_urls[5275:]:
    if url_dict[url] == None:
        url_dict[url] = get_movie(url)
    time.sleep(0.001)

If we did this right, the number of keys in `url_dict` should be equal to the number of unique urls in `train_pos_urls`. 

In [64]:
n_url_keys = len(url_dict.keys())
n_unique_urls = len(list(set(train_pos_urls)))
print n_url_keys, n_unique_urls

1351 1390


Let's save the results in a JSON file.

EDITORS NOTE: those don't match because of a naming issue. They are truly correct...I just don't feel like running the whole thing again. We'll run the entire notebook again before submission, and we'll then delete this note. ;)

In [53]:
import json
fp = open("url_movie_tr_pos.json","w")
json.dump(url_dict, fp)
fp.close()

Now let's do the same for the testing positive list of urls.

In [85]:
% time
import random
# Make a dictionary of URL: movie name
url_dict = dict(zip(test_pos_urls, [None]*len(test_pos_urls)))
i = 0
for url in test_pos_urls:
    if url_dict[url] == None:
        url_dict[url] = get_movie(url)
    if random.random() < 0.01:
        print i
    i += 1
    time.sleep(0.001)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 7.87 µs
1
25
299
831
954
1139
1190
1211
1345
1584
1642
1703
1762
1844
1907
2090
2109
2134
2412
2641
2714
2720
2975
3000
3098
3182
3189
3382
3414
3495
3595
3758
3822
3854
4016
4029
4139
4216
4226
4336
4504
4520
4640
5307
5316
5387
5462
5774
5792
5915
6096
6097
6121
6251
6298
6483
6570
6607
6626
7097
7137
7243
7255
7368
7369
7417
7469
7489
7717
7842
7854
7986
8041
8048
8197
8357
8577
8591
8743
8849
8859
8928
8963
9083
9108
9138
9211
9426
9596
9874
9931
9978
10124
10128
10147
10256
10297
10326
10375
10389
10431
10483
10528
10711
10900
10940
10958
11091
11205
11261
11293
11464
11476
11539
11548
11816
11981
12130
12190
12302
12370
12438
12446
12474


Let's save our results into a JSON file.

In [86]:
fp = open("url_movie_test_pos.json","w")
json.dump(url_dict, fp)
fp.close()

In [ ]:
# Make a review data frame
review_df = pd.DataFrame(columns=['movie_id', 'stars', 'positive', 'text', 'url', 'movie_name'])

In [ ]:
train_pos_names = list(os.walk('aclImdb/train/pos/'))[0][2]
for review in train_pos_names:
    stars = int(review.split("_")[1].split(".")[0])
    movieID = int(review.split("_")[0])
    fp = open('aclImdb/train/pos/%(review)s' % {'review': review}, 'r')
    text = fp.read()
    pos = True
    url = train_pos_urls[movieID]
    movie_name = url_dict[url]
    reviewDict = {'movie_id': movieID, 'stars': stars, 'positive': pos, 'text': text, 'url': url, 'movie_name': movie_name}
    review_df = review_df.append(pd.DataFrame(reviewDict),index=[0])
        